# Generation of LCA indicators and associated .mod and .dat files

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import bw2data as bd
from mescal import *
from energyscope.models import Model
from energyscope.energyscope import Energyscope
from energyscope.result import postprocessing

In [3]:
ei_version = '3.10'
year = 2050

In [4]:
path_inputs = '../01_Notebooks/Data/'
path_model = '../02_Model/'
path_results = f'../03_Results/LCA/{year}/'

## Initialize the EnergyScope model

In [5]:
# AMPL licence 
path_to_ampl_licence = r'C:\Users\matth\ampl' # Path to the AMPL licence file
os.environ['PATH'] = path_to_ampl_licence+':'+os.environ['PATH']

In [6]:
# Define the solver options
solver_options = {
    'solver': 'cplex',
    'presolve_eps': 5e-9,
    'presolve_fixeps': 7e-10,
    'mipgap': 1e-10,
    'display_eps': 1e-10,
    'omit_zero_rows': 1,
    'omit_zero_cols': 1,
    'show_stats': 1,
    'solver_msg': 1,
    'cplex_options': 'integrality=5e-7'
}

In [7]:
# Initialize the QC model with .mod and .dat files
model = Model(
    mod_files=[
        path_model+'QC_es_main.mod',
    ],
    dat_files=[
        path_model+'QC_data.dat',
        path_model+'QC_mob_techs_dist_B2D.dat',
        path_model+'QC_techs_B2D.dat',
        path_model+'QC_mob_params.dat',
    ],
)

In [8]:
# Initialize the EnergyScope model
es = Energyscope(model=model, solver_options=solver_options)

In [9]:
# Solve the model and get results
results = postprocessing(es.calc())


Presolve eliminates 13841 constraints and 10034 variables.
Adjusted problem:
16731 variables:
	822 binary variables
	15909 linear variables
22585 constraints, all linear; 113409 nonzeros
	10356 equality constraints
	12229 inequality constraints
0 objectives.

Setting $presolve_fixeps < 4.41e-10 or >= 5.54e-09
could change presolve results.

CPLEX 20.1.0.0: integrality=5e-7
CPLEX 20.1.0.0: optimal integer solution; objective 0
7284 MIP simplex iterations
0 branch-and-bound nodes
Objective = find a feasible point.


# Generate the LCA indicators 

## Load data and initialize the ESM class

In [10]:
# Load the data
mapping = pd.read_csv(path_inputs+f'mapping_{ei_version}.csv')
unit_conversion = pd.read_excel(path_inputs+'unit_conversion.xlsx')  # open file and press enter in one computation cell to avoid misreading
techno_compositions = pd.read_csv(path_inputs+'technology_compositions.csv')
tech_specifics = pd.read_csv(path_inputs+'technology_specifics.csv')
efficiency = pd.read_csv(path_inputs+'efficiency.csv')
lifetime = pd.read_csv(path_inputs+'lifetime.csv')
mapping_es_flows_to_cpc = pd.read_csv(path_inputs+'mapping_esm_flows_to_CPC.csv')
impact_abbrev = pd.read_csv(path_inputs+'impact_abbrev.csv')

In [11]:
# Load the model from energyscope model
model = results.parameters['layers_in_out'].reset_index().rename(columns={'index0':'Name', 'index1':'Flow', 'layers_in_out':'Amount'}).drop(columns=['Run'])
model = model[model['Amount'] != 0]

In [12]:
# Databases names 
name_main_database = f'ecoinvent_cutoff_{ei_version}_image_SSP2-Base_{year}+truck_carculator regionalized'
name_spatialized_biosphere_db = 'biosphere3_spatialized_flows'
name_es_database = f'EnergyScope_CA-QC_{year}'

In [13]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ei_version}')

In [14]:
main_db = Database(name_main_database)

Loaded ecoinvent_cutoff_3.10_image_SSP2-Base_2050+truck_carculator regionalized from pickle!


In [15]:
ranking_best_ecoinvent_locations_for_QC = [
    'CA-QC', # Quebec
    'CA', # Canada
    'CAN', # Canada in IMAGE
    'CAZ', # Canada - Australia - New Zealand in REMIND
    'RNA', # North America
    'US', # United States
    'USA', # United States in REMIND and IMAGE
    'GLO', # Global average 
    'RoW', # Rest of the world
]

In [16]:
# Add CPC categories to the main database
main_db.add_CPC_categories()

In [17]:
# Change the main database name if needed
if mapping['Database'].iloc[0] != name_main_database:
    mapping['Database'] = len(mapping) * [name_main_database]
missing_flows = main_db.test_mapping_file(mapping)

Mapping successfully linked to the database


In [18]:
esm = ESM(
    # Mandatory inputs
    mapping=mapping,
    unit_conversion=unit_conversion,
    model=model,
    mapping_esm_flows_to_CPC_cat=mapping_es_flows_to_cpc,
    main_database=main_db,
    esm_db_name=name_es_database,
    
    # Optional inputs
    technology_compositions=techno_compositions,
    tech_specifics=tech_specifics,
    lifetime=lifetime,
    efficiency=efficiency,
    regionalize_foregrounds=True,
    accepted_locations=['CA-QC', 'CA'],
    locations_ranking=ranking_best_ecoinvent_locations_for_QC,
    esm_location='CA-QC',
    results_path_file=path_results,
    
    # If we want regionalized results 
    spatialized_biosphere_db=Database(name_spatialized_biosphere_db),
)

Getting activity data


100%|██████████| 88770/88770 [00:01<00:00, 66049.14it/s] 


Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|██████████| 88770/88770 [00:00<00:00, 2582331.94it/s]


Loaded biosphere3_spatialized_flows from brightway!


In [19]:
esm.check_inputs()

List of technologies or resources that are in the model file but not in the mapping file. Their impact scores will be set to the default value.

--> ['CO2_E', 'DEC_RENOVATION', 'DHN_RENOVATION', 'DIESEL_S', 'ELEC_S', 'GASOLINE_S', 'HT_LT', 'LT_DEC_WH', 'LT_DHN_WH', 'NG_S', 'RES_GEO', 'RES_HYDRO', 'RES_SOLAR', 'RES_TIDAL', 'RES_WIND_OFFSHORE', 'RES_WIND_ONSHORE', 'STO_CO2', 'STO_DIE', 'STO_ELEC', 'STO_GASO', 'STO_H2', 'STO_NG']

List of technologies or resources that are in the mapping file but not in the model file (this will not be a problem in the workflow).

--> ['BATTERY', 'DEC_TH_STORAGE', 'DHN_TH_STORAGE', 'EHV_GRID', 'HV_GRID', 'LCV_FC_CH4_MD', 'LCV_FC_CH4_SD', 'LV_GRID', 'MV_GRID']

List of technologies that are in the tech_specifics file but not in the mapping file (this will not be a problem in the workflow).

--> ['AEC_OG', 'MOB_FREIGHT_ELD', 'MOB_FREIGHT_LD', 'MOB_FREIGHT_MD', 'MOB_FREIGHT_SD', 'MOB_PRIVATE_ELD', 'MOB_PRIVATE_LD', 'MOB_PRIVATE_MD', 'MOB_PRIVATE_SD', 'MOB_PU

In [20]:
# Adapt mapping file to ESM location
esm.change_location_mapping_file()
esm.mapping.to_csv(path_inputs+f'mapping_{ei_version}.csv', index=False)

### Generate ESM database

In [21]:
# Foreground regionalization, double-counting removal, and efficiency harmonization
esm.create_esm_database()

### Starting to add construction and resource activities database ###
No location found in your ranking for waste polystyrene - market group for waste polystyrene
--> Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste polyvinylchloride - market group for waste polyvinylchloride
--> Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste plastic, mixture - market group for waste plastic, mixture
--> Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste polyethylene - market group for waste polyethylene
--> Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste glass - market group for waste glass
--> Have to keep the initial location: RER
No location found in your ranking for iridium - platinum group metal, extraction and refinery operations
--> Have to keep the initial location: ZA

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Title: Writing activities to SQLite3 database:
  Started: 12/06/2024 14:33:54
  Finished: 12/06/2024 14:33:56
  Total time elapsed: 00:00:02
  CPU %: 46.50
  Memory %: 12.98
EnergyScope_CA-QC_2050 written to Brightway!
### Database written ###
--> Time: 516.0 seconds


In [22]:
# Save the mapping file with new codes for later use
esm.mapping.to_csv(path_results+f'mapping_{ei_version}_with_new_codes.csv', index=False)

## Generate LCA metrics 

In [23]:
# If the ESM database is already created
# esm.mapping = pd.read_csv(path_results+f'mapping_{ei_version}_with_new_codes.csv')

In [24]:
methods = ['IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10', 'IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10']

### Life-cycle emissions

In [25]:
# Lifetime harmonization
R_long = esm.compute_impact_scores(methods=methods)

Getting activity data


100%|██████████| 1385/1385 [00:00<00:00, 88498.21it/s]


Adding exchange data to activities


100%|██████████| 35626/35626 [00:02<00:00, 17336.89it/s]


Filling out exchange data


100%|██████████| 1385/1385 [00:01<00:00, 928.58it/s] 


Loaded EnergyScope_CA-QC_2050 from brightway!


In [26]:
R_long.to_csv(f'{path_results}impact_scores.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]

### Territorial emissions

In [27]:
activities_subject_to_double_counting = pd.read_csv(f'{path_results}activities_subject_to_double_counting.csv')

In [28]:
R_long_direct_emissions = esm.compute_impact_scores(
    methods=methods,
    assessment_type='direct emissions',
    activities_subject_to_double_counting=activities_subject_to_double_counting,
)

Getting activity data


100%|██████████| 1385/1385 [00:00<00:00, 200930.82it/s]


Adding exchange data to activities


100%|██████████| 35626/35626 [00:02<00:00, 17696.23it/s]


Filling out exchange data


100%|██████████| 1385/1385 [00:01<00:00, 777.33it/s] 


Loaded EnergyScope_CA-QC_2050 from brightway!


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 12/06/2024 15:02:23
  Finished: 12/06/2024 15:02:25
  Total time elapsed: 00:00:01
  CPU %: 61.80
  Memory %: 7.36
EnergyScope_CA-QC_2050_direct_emissions written to Brightway!


In [29]:
R_long_direct_emissions.to_csv(f'{path_results}impact_scores_direct_emissions.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]

## Create the .mod and .dat files

In [30]:
# To skip the impact assessment step
# R_long = pd.read_csv(f'{path_results}impact_scores.csv')
# R_long_direct_emissions = pd.read_csv(f'{path_results}impact_scores_direct_emissions.csv')

In [31]:
metadata = {
    'ecoinvent_version': ei_version,
    'year': year,
    'iam': 'image',
    'ssp_rcp': 'SSP2-Base',
}

In [32]:
# Create .dat file
esm.normalize_lca_metrics(
    R=R_long,
    mip_gap=1e-6,
    lcia_methods=methods,
    specific_lcia_abbrev=['TTHH', 'TTEQ', 'm_CCS'],
    impact_abbrev=impact_abbrev,
    path=path_model,
    metadata=metadata,
    file_name=f'QC_techs_lca_{year}',
)

In [33]:
# Create .dat file for direct emissions only
esm.normalize_lca_metrics(
    assessment_type='direct emissions',
    R=R_long_direct_emissions,
    mip_gap=1e-6,
    lcia_methods=methods,
    specific_lcia_abbrev=['TTHH', 'TTEQ', 'm_CCS'],
    impact_abbrev=impact_abbrev,
    path=path_model,
    metadata=metadata,
    file_name=f'QC_techs_lca_{year}_direct',
)

In [34]:
# Create the .mod file
esm.generate_mod_file_ampl(
    lcia_methods=methods,
    impact_abbrev=impact_abbrev,
    specific_lcia_abbrev=['TTHH', 'TTEQ', 'm_CCS'],
    path=path_model,
    metadata=metadata,
    file_name=f'QC_objectives_lca_{year}',
)

In [35]:
# Create the .mod file for direct emissions only
esm.generate_mod_file_ampl(
    assessment_type='direct emissions',
    lcia_methods=methods,
    impact_abbrev=impact_abbrev,
    specific_lcia_abbrev=['TTHH', 'TTEQ', 'm_CCS'],
    path=path_model,
    metadata=metadata,
    file_name=f'QC_objectives_lca_{year}_direct',
)